In [1]:
from bsml_sat import *

# Prototype BSML Reasoner

This notebook illustrates a prototype for a reasoner for BSML (Bilateral State-based Modal Logic)—see, e.g., the paper ['State-based Modal Logics for Free Choice'](https://arxiv.org/pdf/2305.11777) (Aloni, Anttila & Yang 2024). The reasoning algorithm is based on answer set programming (ASP).

We will first describe the general working of the reasoning algorithm. Then we will see how to invoke it, using a number of example formulas—taken from the paper ['Logic and conversation: The case of free choice'](https://doi.org/10.3765/sp.15.5) (Aloni 2023, Section 9). Finally, we will describe a number of research directions that can be taken to improve this reasoner, many of which are particularly apt for (smaller and larger) student research projects.

## General working
- The reasoner is given two BSML formulas $\varphi_1$ and $\varphi_2$, and the aim is to find a model-state pair $M,s$ that supports $\varphi_1$ and that does not support $\varphi_2$—that is, $M,s \models \varphi_1$ and $M,s \not\models \varphi_2$. Note, this latter condition is not the same as $M,s$ antisupporting $\varphi_2$.
  - This can be used to check whether $\varphi_1 \models \varphi_2$—i.e., this is the case if and only if there is no (suitable) pair $M,s$ such that $M,s \models \varphi_1$ and $M,s \not\models \varphi_2$.
- Two ASP solvers (clingo) working together, in CEGAR-style (*Counterexample-Guided Abstraction Refinement*).
  - The first solver produces pairs $M,s$ (within certain bounds) that support $\varphi_1$.
  - Whenever such a pair $M,s$ with $M,s \models \varphi_1$ is found, this pair $M,s$ is passed to the second solver, which checks whether $M,s \models \varphi_2$.
  - If the second solver finds that $M,s \models \varphi_2$, a new constraint is passed to the first solver ruling out (this particular value for $M,s$ and the search for the first solver continues.
  - If the second solver stops with concluding that $M,s \not\models \varphi_2$, the search stops and $M,s$ is a solution.
- The encoding (for the first solver) uses symmetry breaking constraints that rule out many solutions that are isomorphic. (This helps speed up the algorithm for cases where $\varphi_1 \models \varphi_2$. These symmetry breaking constraints are based on those in the paper ['Symmetry-Breaking Constraints for Directed Graphs'](doi.org/10.3233/FAIA240998) (Rintanen & Rankooh 2024).

## Demonstration
### Some BSML formulas
Let's start by collecting some BSML formulas that we'll use to call the reasoner on.

In [2]:
form0 = Or(Var(1),Var(2))
form1 = Diamond(Or(Var(1),Var(2)))
form2 = Or(Diamond(Var(1)),Diamond(Var(2)))
form3 = And(Diamond(Var(1)),Diamond(Var(2)))
form4 = Neg(form1)
form5 = Neg(form4)
form6 = And(Neg(Diamond(Var(1))),Neg(Diamond(Var(2))))
form7 = Diamond(Neg(And(Var(1),Var(2))))
form8 = And(Diamond(Neg(Var(1))),Diamond(Neg(Var(2))))
form9 = Diamond(Or(Neg(Var(1)), Neg(Var(2))))
form10 = Diamond(Neg(And(Var(1), Var(2))))

### Narrow Scope FC
The narrow scope free-choice entailment corresponds to the following formulas. The reasoner finds that there is no solution (with suitably high values for the number of worlds and variables).

In [3]:
sat_formula = enrich_formula(form1)
unsat_formula = form3
custom_program = ""
# unsat; narrow scope fc entailment

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

Finished the search within time limit..

.. Done solving ..
Total solving time: 16.87 sec


### Wide Scope FC
For wide scope free-choice, the entailment doesn't hold in general, corresponding to a solution.

In [4]:
sat_formula = enrich_formula(form2)
unsat_formula = form3
custom_program = ""
# sat; wide scope fc non-entailment (for arbitrary R)

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=4,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

## Solution ##
relation(3,4)
relation(4,3)
state(3)
state(4)
valuation(3,1)
valuation(4,2)
world(1)
world(2)
world(3)
world(4)
Finished the search within time limit..

.. Done solving ..
Total solving time: 0.07 sec


When we require the relation $R$ to be indisputable in $M,s$, the entailment holds, so there is no solution.

In [5]:
sat_formula = enrich_formula(form2)
unsat_formula = form3
custom_program = """
    % Require that R is indisputable in the given model
    :- state(R,W1), state(R,W2), ft_root(R),
        relation(W1,W3), not relation(W2,W3).
"""
# unsat; wide scope fc non-entailment (for indisputable R)

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

Finished the search within time limit..

.. Done solving ..
Total solving time: 10.91 sec


### Dual prohibition

For dual prohibition, we again have entailment, so no solution.

In [6]:
sat_formula = enrich_formula(form4)
unsat_formula = form6
custom_program = ""
# unsat; dual prohibition entailment

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

Finished the search within time limit..

.. Done solving ..
Total solving time: 7.93 sec


### Double negation

For double negation, we again have entailment, so no solution.

In [7]:
sat_formula = enrich_formula(form5)
unsat_formula = form3
custom_program = ""
# unsat; double negation entailment

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

Finished the search within time limit..

.. Done solving ..
Total solving time: 17.35 sec


### Modal disjunction

For modal disjunction, in general the entailment doesn't hold, so we get a solution.

In [8]:
sat_formula = enrich_formula(form0)
unsat_formula = form3
custom_program = ""
# sat; modal disjunction non-entailment (for arbitrary R)

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=4,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

## Solution ##
state(1)
state(4)
valuation(1,2)
valuation(4,1)
world(1)
world(2)
world(3)
world(4)
Finished the search within time limit..

.. Done solving ..
Total solving time: 0.00 sec


However, when we require the relation $R$ to be state-based in $M,s$, the entailment holds, so there is no solution.

In [9]:
sat_formula = enrich_formula(form0)
unsat_formula = form3
custom_program = """
    % Require that R is state-based in the given model
    :- state(R,W1), ft_root(R),
        relation(W1,W2), not state(R,W2).
    :- state(R,W1), ft_root(R),
        not relation(W1,W2), state(R,W2).
"""
# unsat; modal disjunction entailment (for state-based R)

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

Finished the search within time limit..

.. Done solving ..
Total solving time: 0.80 sec


### Detachability

For detachability, again, no entailment so no solution.

In [10]:
sat_formula = enrich_formula(form7)
unsat_formula = form8
custom_program = ""
# sat; detachability non-entailment

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

## Solution ##
relation(3,2)
state(3)
valuation(2,2)
world(1)
world(2)
world(3)
Finished the search within time limit..

.. Done solving ..
Total solving time: 0.01 sec


### Some logical equivalences

When taking a logical equivalence, entailments in both directions hold,
so in both cases, we get no solution.

In [11]:
sat_formula = form9
unsat_formula = form10
custom_program = ""
# unsat; validity

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

Finished the search within time limit..

.. Done solving ..
Total solving time: 108.39 sec


In [12]:
sat_formula = form10
unsat_formula = form9
custom_program = ""
# unsat; validity

solve_bsml_sat(
    sat_formula,
    unsat_formula,
    max_num_worlds=3,
    num_vars=3,
    timeout=300,
    custom_program=custom_program,
    use_minimization_heuristics=True,
)

.. Grounding ..
.. Solving ..

Finished the search within time limit..

.. Done solving ..
Total solving time: 94.90 sec


## Research directions

This implementation of a BSML reasoner is only a prototype, and much remains to be explored and done. Here are some possible interesting and/or useful research directions.

### More implementational directions
- Collect a (structured) set of benchmark inputs to test the reasoner (and future improvements) on
- Develop and describe a (text-based) file format for storing/loading BSML formulas
- Develop this prototype reasoner into a proper Python package that can easily be installed and used
- Set up a solving competition (for fellow students that want to try improving the reasoner)

### Theoretical directions
- Prove correctness of this approach (based on the two-solver algorithm with the particular encodings used)
- Derive tight bounds on the size of models that need to be considered for the algorithm to be completeness (i.e., if the algorithm finds no $M,s$ within these bounds such that $M,s \models \varphi_1$ and $M,s \not\models \varphi_2$, then it is guaranteed that $\varphi_1 \models \varphi_2$).

### Combined (theoretical and practical/experimental) directions
- Develop more effective symmetry breaking constraints, for example based on a suitable notion of bisimulation. *(Difficulty: high)*
- Develop a dedicated propagator for symmetry breaking to rule out isomorphic models (similar in style to [SAT Modulo Symmetries](https://sat-modulo-symmetries.readthedocs.io/en/latest/)). *(Difficulty: high)*
- Develop better refinement strategies–e.g., stronger constraints that can be added to the first solver that are based on solutions that the second solver finds for some $M,s$. *(Difficulty: high)*
- Experimenting with variants of the encodings used. *(Difficulty: medium)*
- Experimenting with different problem-specific search heuristics. *(Difficulty: lower)*
- Trying multi-shot solving approaches to see if that leads to better performance. *(Difficulty: medium)*
- Developing an entirely different algorithm/encoding that is based on natural deduction proofs for BSML—see, e.g., the paper ['State-based Modal Logics for Free Choice'](https://arxiv.org/pdf/2305.11777) (Aloni, Anttila & Yang 2024). *(Difficulty: high)*